In [1]:
!pip install -U langchain-openai

# LangChain 实战：房产销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的房地产销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
在房地产销售行业中，精心准备的销售话术可以极大地提高成交率。以下是一些适用于中国房地产销售的实用话术：

1.  
[客户问题] 这个小区交通便利吗？
[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。

2.  
[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

3.  
[客户问题] 我看房价还在涨，这个投资回报怎么样？
[销售回答] 这个区域未来有大量的商业和基础设施建设，所以从长期来看，投资回报非常有保证。

4.  
[客户问题] 有没有学校？
[销售回答] 附近有多所优质的学校，非常适合有孩子的家庭。

5.  
[客户问题] 物业管理怎么样？
[销售回答] 我们的物业管理得到了业主一致好评，服务非常到位。

6.  
[客户问题] 我想要南向的房子。
[销售回答] 很好，我们确实有一些朝南的单位，它们的采光特别好。

7.  
[客户问题] 这个小区安全吗？
[销售回答] 当然，我们24小时安保巡逻，还有先进的监控系统。

8.  
[客户问题] 预计什么时候交房？
[销售回答] 根据目前的进度，我们预计将在明年底交房。

9.  
[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。

10.  
[客户问题] 有优惠吗？
[销售回答] 当然，如果您现在下订，我们可以给您一些优惠。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [2]:
with open("fruit_sales_words.txt",encoding='utf-8') as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [3]:
from langchain.text_splitter import CharacterTextSplitter

In [4]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [5]:
docs = text_splitter.create_documents([real_estate_sales])

Created a chunk of size 104, which is longer than the specified 100
Created a chunk of size 101, which is longer than the specified 100
Created a chunk of size 102, which is longer than the specified 100


In [6]:
docs[0]

Document(page_content='**[客户问题]**：你们的水果为什么比别家的贵？\n   **[销售回答]**：我们的水果都是精选自优质产地，保证新鲜和高品质。而且我们的运输和储存条件都是最好的，所以口感和营养价值都非常高，物有所值。')

In [7]:
len(docs)

37

In [8]:
import os

os.environ['HTTP_PROXY'] = 'http://127.0.0.1:10810'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:10810'
print(os.environ.get("OPENAI_API_KEY"));

sk-proj-3qMKwHdieh8eXSmtEajbT3BlbkFJGMcnkLA9dUVxRf4g0seS


### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [9]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

D:\software\Miniconda\Miniconda24.5.0\envs\ITATraining\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [10]:
query = "水果哪个好吃？"

In [11]:
answer_list = db.similarity_search(query)

In [12]:
for ans in answer_list:
    print(ans.page_content + "\n")

**[客户问题]**：这个水果怎么吃比较好？
   **[销售回答]**：您可以直接食用，也可以榨汁、做水果沙拉或者加入酸奶中，都是非常不错的选择。如果您需要食谱，可以向我们咨询。

**[客户问题]**：这些水果怎么挑选比较好？
    **[销售回答]**：我们可以帮您挑选最好的水果，您也可以根据水果的颜色、形状和气味来判断，具体方法我们可以为您详细介绍。

**[客户问题]**：这个水果适合小孩吃吗？
   **[销售回答]**：我们这个水果营养丰富、口感好，非常适合小孩食用，补充维生素和纤维，有助于孩子的健康成长。

**[客户问题]**：这个水果的口感怎么样？
    **[销售回答]**：这个水果的口感非常好，甜而不腻，汁水丰富，非常适合直接食用。



In [13]:
db.save_local("real_estates_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [14]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 2})

In [15]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000022D492E1E70>, search_kwargs={'k': 2})

In [16]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

D:\software\Miniconda\Miniconda24.5.0\envs\ITATraining\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


**[客户问题]**：这个水果怎么吃比较好？
   **[销售回答]**：您可以直接食用，也可以榨汁、做水果沙拉或者加入酸奶中，都是非常不错的选择。如果您需要食谱，可以向我们咨询。

**[客户问题]**：这些水果怎么挑选比较好？
    **[销售回答]**：我们可以帮您挑选最好的水果，您也可以根据水果的颜色、形状和气味来判断，具体方法我们可以为您详细介绍。



In [17]:
docs = topK_retriever.get_relevant_documents("凤梨和菠萝哪个更好吃。")

In [18]:
for doc in docs:
    print(doc.page_content + "\n")

**[客户问题]**：这个水果怎么吃比较好？
   **[销售回答]**：您可以直接食用，也可以榨汁、做水果沙拉或者加入酸奶中，都是非常不错的选择。如果您需要食谱，可以向我们咨询。

**[客户问题]**：这些水果怎么挑选比较好？
    **[销售回答]**：我们可以帮您挑选最好的水果，您也可以根据水果的颜色、形状和气味来判断，具体方法我们可以为您详细介绍。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [19]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.5}
)

In [35]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

**[客户问题]**：水果的价格还可以再优惠一点吗？
    **[销售回答]**：我们的价格已经是最优惠的了，而且我们提供的品质和服务绝对值得这个价格，您可以放心购买。

**[客户问题]**：这个水果为什么这么贵？
    **[销售回答]**：这个水果是从优质产地精挑细选的，而且运输和储存成本较高，价格虽然稍高，但品质绝对值得这个价格。

**[客户问题]**：能不能尝一下？
   **[销售回答]**：当然可以，我们这里有提供试吃，您可以先尝一下，满意再购买。

**[客户问题]**：你们这里有没有优惠活动？
    **[销售回答]**：我们经常会有各种优惠活动，比如节假日特价、会员专享折扣等，您可以关注我们的公众号或加入我们的会员，随时获取最新优惠信息。



### 提取向量数据库中的`销售回答`

In [51]:
docs = retriever.get_relevant_documents(query)

In [52]:
docs[0].page_content

'**[客户问题]**：水果的价格还可以再优惠一点吗？\n    **[销售回答]**：我们的价格已经是最优惠的了，而且我们提供的品质和服务绝对值得这个价格，您可以放心购买。'

In [53]:
docs[0].page_content.split("**[销售回答]**：")

['**[客户问题]**：水果的价格还可以再优惠一点吗？\n    ',
 '我们的价格已经是最优惠的了，而且我们提供的品质和服务绝对值得这个价格，您可以放心购买。']

In [54]:
ans = docs[0].page_content.split("**[销售回答]**：")[-1]

In [55]:
ans

'我们的价格已经是最优惠的了，而且我们提供的品质和服务绝对值得这个价格，您可以放心购买。'

#### 尝试各种问题

In [56]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("**[销售回答]**：")[-1] for doc in docs]

    return ans_list

In [60]:
query = "这个水果怎么卖？"

print(sales(query))

['这个水果是从优质产地精挑细选的，而且运输和储存成本较高，价格虽然稍高，但品质绝对值得这个价格。', '这个水果的口感非常好，甜而不腻，汁水丰富，非常适合直接食用。']


In [61]:
print(sales(query, 0.8))

['这个水果是从优质产地精挑细选的，而且运输和储存成本较高，价格虽然稍高，但品质绝对值得这个价格。', '这个水果的口感非常好，甜而不腻，汁水丰富，非常适合直接食用。']


In [62]:
query = "价格200万以内"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

D:\software\Miniconda\Miniconda24.5.0\envs\ITATraining\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []



D:\software\Miniconda\Miniconda24.5.0\envs\ITATraining\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


score:0.75 ans: []

score:0.5 ans: ['我们的价格已经是最优惠的了，而且我们提供的品质和服务绝对值得这个价格，您可以放心购买。', '这个水果是从优质产地精挑细选的，而且运输和储存成本较高，价格虽然稍高，但品质绝对值得这个价格。', '当然可以，我们这里有提供试吃，您可以先尝一下，满意再购买。', '我们经常会有各种优惠活动，比如节假日特价、会员专享折扣等，您可以关注我们的公众号或加入我们的会员，随时获取最新优惠信息。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [63]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [64]:
qa_chain({"query": "你们小区有200万的房子吗？"})

D:\software\Miniconda\Miniconda24.5.0\envs\ITATraining\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '你们小区有200万的房子吗？',
 'result': "I'm sorry, but as an AI language model, I don't have access to real-time data or information about housing prices in specific locations. If you're interested in purchasing a property, I would recommend checking real estate listings online, visiting a local real estate agency, or contacting property developers in the area you're interested in to get the most up-to-date information on available properties and their prices."}

In [65]:
qa_chain({"query": "小区吵不吵"})

D:\software\Miniconda\Miniconda24.5.0\envs\ITATraining\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '小区吵不吵',
 'result': '"小区吵不吵" 这个问题我无法直接回答，因为这取决于具体的小区环境和个人感受。不同的小区有不同的噪音水平，有些可能靠近繁忙的道路或商业区，可能会比较吵；而有些可能位于较为安静的住宅区或者有良好的隔音措施，相对会比较安静。如果你想知道某个具体小区的噪音情况，可能需要询问那个小区的居民或者亲自去那里体验一下。'}

In [66]:
print(sales("小区吵不吵"))

[]


D:\software\Miniconda\Miniconda24.5.0\envs\ITATraining\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


## 加载 FAISS 向量数据库已有结果

In [68]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings(),allow_dangerous_deserialization=True)

In [72]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [73]:
qa_chain({"query": "我想买别墅，你们有么"})

D:\software\Miniconda\Miniconda24.5.0\envs\ITATraining\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '我想买别墅，你们有么',
 'result': '对不起，作为一个人工智能，我无法提供购买别墅的服务。我主要用于提供信息和解答问题。如果你有关于别墅的问题，我会很乐意帮助你。'}

In [74]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [75]:
qa_chain({"query": "我想买别墅，你们有么"})

D:\software\Miniconda\Miniconda24.5.0\envs\ITATraining\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买别墅，你们有么',
 'result': '对不起，我是一个人工智能助手，无法提供购买别墅的服务。我主要用来提供信息和解答问题。如果你有其他问题，我会尽力帮助你。'}

In [76]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [77]:
result = qa_chain({"query": "我想买别墅，你们有么"})

D:\software\Miniconda\Miniconda24.5.0\envs\ITATraining\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


In [78]:
result

{'query': '我想买别墅，你们有么',
 'result': '对不起，作为一个AI，我无法提供购买别墅的服务。我建议你联系一个房地产经纪人或者搜索在线房地产平台以获取帮助。',
 'source_documents': []}